In [57]:
number_pad = [[7,8,9], [4,5,6], [1,2,3], ['X', 0, 'A']]
direction_pad = [['X', '^', 'A'], ['<', 'v', '>']]

In [58]:
directions = {(0, 1): '>', (0, -1): '<', (1, 0): 'v', (-1, 0): '^'}

In [59]:
memo = {}

In [60]:
simple_memo = {}

In [61]:
def find_possible_paths_simple(start, end, pad):
    if (start, end) in simple_memo:
        return simple_memo[(start, end)]
    for y, row in enumerate(pad):
        for x, elem in enumerate(row):
            if elem == start:
                start_pos = (y, x)
            if elem == end:
                end_pos = (y, x)
    possible_paths = []
    traverse(pad, start_pos, end_pos, possible_paths, [])
    least_changes = 99999999999
    paths_by_changes = {}
    for path in possible_paths:
        if len(path) == 1:
            memo[(start, end)] = path
            return path
        else:
            changes = 0
            current = path[0]
            for elem in path[1:]:
                if elem != current:
                    changes += 1
                current = elem
            if changes < least_changes:
                least_changes = changes
                paths_by_changes[least_changes] = path
    simple_memo[(start, end)] = paths_by_changes[least_changes]
    return paths_by_changes[least_changes]

In [62]:
def traverse(pad, start_pos, end_pos, possible_paths, path=[]):
    path.append(start_pos)
    if start_pos == end_pos:
        dir_path = []
        for i, elem in enumerate(path):
            if i == len(path) - 1:
                dir_path.append('A')
                break
            dir = (path[i+1][0] - elem[0], path[i+1][1] - elem[1])
            dir_path.append(directions[dir])
        possible_paths.append(dir_path)
        return
    for dir in directions:
        new_pos = (start_pos[0] + dir[0], start_pos[1] + dir[1])
        if 0 <= new_pos[0] < len(pad) and 0 <= new_pos[1] < len(pad[0]) and pad[new_pos[0]][new_pos[1]] != 'X' and new_pos not in path:
            traverse(pad, new_pos, end_pos, possible_paths, path[:])

def find_possible_paths(start, end, pad):
    if (start, end) in memo:
        return memo[(start, end)]
    for y, row in enumerate(pad):
        for x, elem in enumerate(row):
            if elem == start:
                start_pos = (y, x)
            if elem == end:
                end_pos = (y, x)
    possible_paths = []
    traverse(pad, start_pos, end_pos, possible_paths, [])
    least_changes = 99999999999
    paths_by_changes = {}
    for path in possible_paths:
        if len(path) == 1:
            memo[(start, end)] = [path]
            return [path]
        else:
            changes = 0
            current = path[0]
            for elem in path[1:]:
                if elem != current:
                    changes += 1
                current = elem
            if changes <= least_changes:
                least_changes = changes
                if not least_changes in paths_by_changes:
                    paths_by_changes[least_changes] = []
                paths_by_changes[least_changes].append(path)
    memo[(start, end)] = paths_by_changes[least_changes]
    return paths_by_changes[least_changes]


In [63]:
filename = 'input.txt'

with open(filename) as f:
    codes = [['A'] + list(line.strip()) for line in f.readlines()]
codes

[['A', '1', '4', '0', 'A'],
 ['A', '1', '7', '0', 'A'],
 ['A', '1', '6', '9', 'A'],
 ['A', '8', '0', '3', 'A'],
 ['A', '1', '2', '9', 'A']]

In [64]:
def shortest_array(arrays):
    shortest_len = None
    for arr in arrays:
        if not shortest_len or len(arr) < shortest_len:
            shortest_len = len(arr)
    return shortest_len

In [65]:
def recursive_path_finder(start, end, depth, end_depth, first):
    key = f'{start}-{end}-{depth}'
    if key in memo_len and not first:
        return memo_len[key]
    if first:
        path_starts = find_possible_paths('A', start, direction_pad)
    possible_paths = find_possible_paths(start, end, direction_pad)

    if first:
        possible_total_paths = []
        for path_start in path_starts:
            for possible_path in possible_paths:
                possible_total_paths.append(path_start[:]+possible_path[:])
    else:
        possible_total_paths = possible_paths

    if depth == end_depth:
        shortest_route_length = shortest_array(possible_total_paths)
        if not first:
            memo_len[key] = shortest_route_length
        return shortest_route_length

    shortest_length = None

    for possible_path in possible_total_paths:

        if len(possible_path) == 1 and possible_path[0] == 'A':
            if not first:
                memo_len[key] = 1
            return 1

        sum_length = 0
        for i, elem in enumerate(possible_path):
            if i == len(possible_path) - 1:
                break
            sum_length += recursive_path_finder(elem, possible_path[i + 1], depth + 1, end_depth, True if i == 0 else False)

        if not shortest_length or sum_length < shortest_length:
            shortest_length = sum_length


    if not first:
        memo_len[key] = shortest_length
    return shortest_length

In [66]:
path = []

memo_len = {}

def possible_int(elem):
    return int(elem) if elem != 'A' else elem

index = 0
optional_paths = {}
for index in range(len(codes)):
    path = []
    for i, elem in enumerate(codes[index]):
        if i == len(codes) - 1:
            break
        next_elem = codes[index][i + 1]
        start = possible_int(elem)
        end = possible_int(next_elem)
        optional_paths[i] = find_possible_paths(start, end, number_pad)

    paths = []

    for i in optional_paths:
        temp_paths = []
        for arr in optional_paths[i]:
            if len(paths) == 0:
                temp_paths.append(arr)
            else:
                for path in paths:
                    temp_paths.append(path[:] + arr[:])
        paths = temp_paths

    shortest_option_length = None
    for path_alt in paths:
        path_len = 0
        for i, elem in enumerate(path_alt):
            if i == len(path_alt) - 1:
                break
            path_len += recursive_path_finder(elem, path_alt[i+1], 1, 25, True if i == 0 else False)
        if not shortest_option_length or path_len < shortest_option_length:
            shortest_option_length = path_len
    print(shortest_option_length)


87513499934
87513499936
91059074548
93831469526
90594397580


In [67]:
87513499934 * 140 + 87513499936*170 + 91059074548*169 + 93831469526*803 + 90594397580*129

129551515895690